## Лабораторная Работа 3

### Install dependencies

In [1]:
pip install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zz3jzjdf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-zz3jzjdf
  Resolved https://github.com/huggingface/transformers to commit 9aab965b1e61d92d402809bd467c317ec464e560
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install faster-whisper vosk jiwer librosa soundfile pydub torch datasets torchaudio gtts

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import time
import wave
import json
import numpy as np
import pandas as pd
from faster_whisper import WhisperModel
from vosk import Model, KaldiRecognizer
import jiwer
import librosa
import soundfile as sf
import urllib.request
import zipfile
import shutil

### Download Vosk and Verify Installation

In [4]:
def download_and_setup_vosk():
    model_url = "https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip"
    model_zip = "vosk-model-small-ru-0.22.zip"
    model_dir = "vosk-model-small-ru-0.22"

    
    try:
        # Download the model
        urllib.request.urlretrieve(model_url, model_zip)
        print("Download complete")
        
        # Extract the model
        with zipfile.ZipFile(model_zip, 'r') as zip_ref:
            zip_ref.extractall(".")
        print("Extraction complete")
        
        # Clean up zip file
        os.remove(model_zip)
        print("Cleanup complete")
        
        # Verify the model structure
        if os.path.exists(model_dir):
            contents = os.listdir(model_dir)
            print(f"Model contents: {contents}")
            
            # Test if model works
            from vosk import Model
            try:
                test_model = Model(model_dir)
                print("VOSK MODEL IS WORKING!")
                print(f"Model path: {os.path.abspath(model_dir)}")
                return True
            except Exception as e:
                print(f"Model test failed: {e}")
                return False
        else:
            print("Model directory not created")
            return False
            
    except Exception as e:
        print(f"Download failed: {e}")
        return False

if __name__ == "__main__":
    success = download_and_setup_vosk()
    if success:
        print("\nREADY! You can now run your main ASR comparison.")
    else:
        print("\nFailed to set up Vosk model.")

Download complete
Extraction complete
Cleanup complete
Model contents: ['README', 'ivector', 'am', 'graph', 'conf']


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-ru-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-ru-0.22/graph/HCLr.fst vosk-model-small-ru-0.22/graph/Gr.fst


VOSK MODEL IS WORKING!
Model path: /home/nyathi/AudioSignals/Lab3/vosk-model-small-ru-0.22

READY! You can now run your main ASR comparison.


LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo vosk-model-small-ru-0.22/graph/phones/word_boundary.int


### Checking GigaAM installation

In [5]:
import gigaam
model = gigaam.load_model("ctc")
print(model)

/home/nyathi/GigaAM/gigaam/__init__.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location="cpu")


GigaAMASR(
  (preprocessor): FeatureExtractor(
    (featurizer): Sequential(
      (0): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
      (1): SpecScaler()
    )
  )
  (encoder): ConformerEncoder(
    (pre_encode): StridingSubsampling(
      (out): Linear(in_features=12288, out_features=768, bias=True)
      (conv): Sequential(
        (0): Conv2d(1, 768, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(768, 768, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): ReLU()
      )
    )
    (pos_enc): RotaryPositionalEmbedding()
    (layers): ModuleList(
      (0-15): 16 x ConformerLayer(
        (norm_feed_forward1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (feed_forward1): ConformerFeedForward(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (activation): SiLU()
          (linear2): Linear(in_features=3072, out_features=768, bi

### Create audio files

In [6]:
'''
from gtts import gTTS
import os

def create_audio_files():
    """Create Russian audio files using Google TTS"""
    
    # Create directories
    os.makedirs("raw_audio", exist_ok=True)
    os.makedirs("reference_texts", exist_ok=True)
    
    # Russian texts for main comparison (10 files)
    main_texts = [
        # Longer, more complex sentences with varied vocabulary
        "В современном мире искусственный интеллект стремительно развивается и находит применение в различных сферах человеческой деятельности, начиная от медицины и заканчивая финансовыми технологиями.",
        
        "Несмотря на значительные достижения в области машинного обучения, создание по-настоящему разумных систем, способных к абстрактному мышлению и творчеству, остается сложнейшей задачей для исследователей.",
        
        "Русский язык, являясь одним из самых богатых и выразительных языков мира, обладает сложной грамматической структурой и многообразием стилистических средств, что представляет особый интерес для лингвистического анализа.",
        
        "Климатические изменения, наблюдаемые в последние десятилетия, оказывают profound влияние на экосистемы по всей планете, вызывая беспокойство у научного сообщества и требуя незамедлительных мер.",
        
        "Экономическая глобализация, с одной стороны, способствует развитию международной торговли и культурному обмену, а с другой — создает определенные challenges для национальных экономик и социальных структур.",
        
        "Квантовые вычисления, основанные на принципах квантовой механики, promise революционизировать области криптографии, молекулярного моделирования и оптимизации сложных систем в ближайшие годы.",
        
        "Философские концепции, разработанные античными мыслителями, продолжают оказывать значительное влияние на современную этику, политическую теорию и методологию научного познания.",
        
        "Биотехнологический прогресс последних лет позволяет редактировать геномы живых организмов с unprecedented точностью, открывая новые возможности в медицине и сельском хозяйстве.",
        
        "Архитектурные памятники древних цивилизаций, сохранившиеся до наших дней, представляют собой не только историческую ценность, но и источник вдохновения для современных зодчих.",
        
        "Психолингвистические исследования демонстрируют, как языковые структуры влияют на процессы восприятия, категоризации и запоминания информации у носителей разных языков."
    ]

    domain_texts = [
        # Technical/scientific texts with complex terminology
        "Трансформерные архитектуры, основанные на механизмах внимания, кардинально изменили подходы к обработке естественного языка, демонстрируя state-of-the-art результаты в задачах машинного перевода и генерации текста.",
        
        "Глубокое обучение с подкреплением, сочетающее в себе нейронные сети и методы динамического программирования, успешно применяется для решения сложных задач управления в robotics и игровых симуляторах.",
        
        "Диффузионные probabilistic модели, генерирующие данные через процесс последовательной денойзинга, показали remarkable результаты в синтезе изображений высокого разрешения и аудиосигналов.",
        
        "Метаобучение или learning-to-learn подходы позволяют моделям быстро адаптироваться к новым задачам на основе limited количества примеров, имитируя способность человека к быстрому обучению.",
        
        "Мультимодальные нейронные сети, способные одновременно обрабатывать и интегрировать информацию из различных источников such as текст, изображения и аудио, открывают новые перспективы для создания более интеллектуальных AI систем."
    ]
    
    print("Creating main audio files with Google TTS...")
    
    # Create main audio files
    for i, text in enumerate(main_texts, 1):
        # Create audio using Google TTS
        tts = gTTS(text=text, lang='ru', slow=False)
        audio_file = f"raw_audio/main_audio_{i}.wav"
        tts.save(audio_file)
        print(f"Created: {audio_file}")
        
        # Save reference text
        with open(f"reference_texts/audio{i}.txt", "w", encoding="utf-8") as f:
            f.write(text)
        print(f" Created: reference_texts/audio{i}.txt")
    
    print("\nCreating domain audio files...")
    
    # Create domain audio files
    for i, text in enumerate(domain_texts, 1):
        # Create audio using Google TTS
        tts = gTTS(text=text, lang='ru', slow=False)
        audio_file = f"raw_audio/domain_audio_{i}.wav"
        tts.save(audio_file)
        print(f"Created: {audio_file}")
        
        # Save reference text
        with open(f"reference_texts/domain{i}.txt", "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Created: reference_texts/domain{i}.txt")
    
    print(f"\n Created {len(main_texts)} main audio files and {len(domain_texts)} domain audio files!")

if __name__ == "__main__":
    create_audio_files()
    
'''

'\nfrom gtts import gTTS\nimport os\n\ndef create_audio_files():\n    """Create Russian audio files using Google TTS"""\n    \n    # Create directories\n    os.makedirs("raw_audio", exist_ok=True)\n    os.makedirs("reference_texts", exist_ok=True)\n    \n    # Russian texts for main comparison (10 files)\n    main_texts = [\n        # Longer, more complex sentences with varied vocabulary\n        "В современном мире искусственный интеллект стремительно развивается и находит применение в различных сферах человеческой деятельности, начиная от медицины и заканчивая финансовыми технологиями.",\n        \n        "Несмотря на значительные достижения в области машинного обучения, создание по-настоящему разумных систем, способных к абстрактному мышлению и творчеству, остается сложнейшей задачей для исследователей.",\n        \n        "Русский язык, являясь одним из самых богатых и выразительных языков мира, обладает сложной грамматической структурой и многообразием стилистических средств, чт

### Use Existing dataset

In [7]:
# create_audio_tts.py
import os
from datasets import load_dataset
import soundfile as sf

def create_audio_files():
    """Create Russian audio files using Common Voice Russian dataset"""
    
    # Create directories
    os.makedirs("raw_audio", exist_ok=True)
    os.makedirs("reference_texts", exist_ok=True)
    
    print("Loading Common Voice Russian dataset...")
    
    try:
        # Use Common Voice Russian dataset
        dataset = load_dataset("mozilla-foundation/common_voice_16_1", "ru", split="train", streaming=True)
        print("Common Voice Russian dataset loaded successfully")
        
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return
    
    # Find Russian samples with audio and text
    russian_samples = []
    for sample in dataset:
        if len(russian_samples) >= 15:
            break
            
        if has_valid_audio(sample) and has_russian_text(sample):
            russian_samples.append(sample)
            print(f"Found Russian sample {len(russian_samples)}")
    
    print(f"Found {len(russian_samples)} suitable Russian samples")
    
    if len(russian_samples) < 15:
        print(f"Error: Need 15 Russian samples but only found {len(russian_samples)}")
        return
    
    # Select 15 samples (10 for main, 5 for domain)
    selected_samples = russian_samples[:15]
    
    print("Creating main audio files...")
    
    # Create main audio files (10 files)
    for i in range(10):
        sample = selected_samples[i]
        text = get_text(sample)
        audio_data = sample['audio']
        
        audio_file = f"raw_audio/main_audio_{i+1}.wav"
        sf.write(audio_file, audio_data['array'], audio_data['sampling_rate'])
        print(f"Created: {audio_file}")
        
        with open(f"reference_texts/audio{i+1}.txt", "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Created: reference_texts/audio{i+1}.txt")
    
    print("\nCreating domain audio files...")
    
    # Create domain audio files (5 files)
    for i in range(5):
        sample = selected_samples[i + 10]
        text = get_text(sample)
        audio_data = sample['audio']
        
        audio_file = f"raw_audio/domain_audio_{i+1}.wav"
        sf.write(audio_file, audio_data['array'], audio_data['sampling_rate'])
        print(f"Created: {audio_file}")
        
        with open(f"reference_texts/domain{i+1}.txt", "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Created: reference_texts/domain{i+1}.txt")
    
    print(f"\nSuccessfully created 10 main audio files and 5 domain audio files!")

def has_valid_audio(sample):
    """Check if sample has valid audio data"""
    if 'audio' not in sample or sample['audio'] is None:
        return False
    
    audio_data = sample['audio']
    if 'array' not in audio_data or audio_data['array'] is None:
        return False
    
    if len(audio_data['array']) == 0:
        return False
    
    return True

def has_russian_text(sample):
    """Check if sample has Russian text"""
    text = get_text(sample)
    if not text:
        return False
    
    # Check for Cyrillic characters
    return any('\u0400' <= char <= '\u04FF' for char in text)

def get_text(sample):
    """Extract text from sample"""
    if 'sentence' in sample and sample['sentence']:
        return sample['sentence'].strip()
    elif 'text' in sample and sample['text']:
        return sample['text'].strip()
    elif 'transcription' in sample and sample['transcription']:
        return sample['transcription'].strip()
    return ""

if __name__ == "__main__":
    create_audio_files()

Loading Common Voice Russian dataset...
Error loading dataset: Dataset 'mozilla-foundation/common_voice_16_1' is a gated dataset on the Hub. You must be authenticated to access it.


### Convert all raw audio to 16kHz mono WAV

In [8]:
def convert_audio_files():
    print("Converting audio to 16kHz mono WAV...")
    
    # Convert main audio files (10 files)
    for i in range(1, 11):
        input_file = f"raw_audio/main_audio_{i}.wav"
        output_file = f"audio_files/audio{i}.wav"
        
        if os.path.exists(input_file):
            audio, sr = librosa.load(input_file, sr=16000, mono=True)
            sf.write(output_file, audio, 16000, subtype='PCM_16')
            print(f"✓ Converted: {output_file}")
    
    # Convert domain audio files (5 files)  
    for i in range(1, 6):
        input_file = f"raw_audio/domain_audio_{i}.wav"
        output_file = f"domain_audio/domain{i}.wav"
        
        if os.path.exists(input_file):
            audio, sr = librosa.load(input_file, sr=16000, mono=True)
            sf.write(output_file, audio, 16000, subtype='PCM_16')
            print(f"✓ Converted: {output_file}")

convert_audio_files()

Converting audio to 16kHz mono WAV...


### Organizing reference texts

In [9]:
import shutil
import os

print("Organizing reference texts...")

# Copy main reference texts
for i in range(1, 11):
    src = f"reference_texts/audio{i}.txt"
    dst = f"audio_files/audio{i}.txt"
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"✓ Copied: {dst}")

# Copy domain reference texts  
for i in range(1, 6):
    src = f"reference_texts/domain{i}.txt"
    dst = f"domain_audio/domain{i}.txt"
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"✓ Copied: {dst}")

print("✓ ALL FILES ORGANIZED!")

Organizing reference texts...
✓ ALL FILES ORGANIZED!


In [10]:

class ASRComparator:
    def __init__(self):
        self.whisper_model = WhisperModel("base", device="cpu", compute_type="int8")
        self.vosk_model = self._init_vosk()
        self.giga_model = self._init_gigaam()
    
    def _init_vosk(self):
        """Initialize Vosk model with comprehensive error handling"""
        model_path = "vosk-model-small-ru-0.22"
        
        # Check if model directory exists
        if not os.path.exists(model_path):
            print(f"Vosk model directory not found: {model_path}")
            return None
        
        # Check if model directory has necessary files
        required_folders = ['am', 'conf', 'graph']
        if not all(os.path.exists(os.path.join(model_path, folder)) for folder in required_folders):
            print(f"Vosk model directory is incomplete: {model_path}")
            print("   Please re-download the model")
            return None
        
        try:
            model = Model(model_path)
            print("Vosk model loaded successfully!")
            return model
        except Exception as e:
            print(f"Failed to load Vosk model: {e}")
            print("   Model path might be incorrect or model files are corrupted")
            return None
    
    def _init_gigaam(self):
        """Initialize GigaAM with the correct method you found"""
        try:
            import gigaam
            print("Loading GigaAM model...")
            model = gigaam.load_model("ctc")
            print("GigaAM model loaded successfully!")
            return model
        except ImportError as e:
            print(f"GigaAM import failed: {e}")
            return None
        except Exception as e:
            print(f"Failed to load GigaAM model: {e}")
            return None

    def calculate_wer_cer(self, reference, hypothesis):
        """Calculate WER and CER using jiwer"""
        # Clean the texts
        reference = reference.strip()
        hypothesis = hypothesis.strip()
        
        if not reference or not hypothesis:
            return 1.0, 1.0  # Return maximum error if empty
        
        # WER (Word Error Rate)
        wer = jiwer.wer(reference, hypothesis)
        
        # CER (Character Error Rate)
        cer = jiwer.cer(reference, hypothesis)
        
        return wer, cer
    
    def whisper_transcribe(self, audio_path, prompt=None):
        """Transcribe using Whisper with optional prompt"""
        start_time = time.time()
        
        try:
            segments, info = self.whisper_model.transcribe(
                audio_path,
                language="ru",
                initial_prompt=prompt,
                beam_size=5
            )
            
            transcription = " ".join([segment.text for segment in segments])
            end_time = time.time()
            
            return transcription.strip(), end_time - start_time
        except Exception as e:
            print(f"Whisper transcription error: {e}")
            return "", 0.0
    
    def vosk_transcribe(self, audio_path):
        """Transcribe using Vosk"""
        start_time = time.time()
        
        try:
            # Read audio file
            wf = wave.open(audio_path, "rb")
            if wf.getnchannels() != 1:
                print(f"Warning: {audio_path} is not mono")
            if wf.getsampwidth() != 2:
                print(f"Warning: {audio_path} is not 16-bit")
            if wf.getframerate() != 16000:
                print(f"Warning: {audio_path} is not 16kHz")
            
            rec = KaldiRecognizer(self.vosk_model, wf.getframerate())
            rec.SetWords(True)
            
            transcription = ""
            while True:
                data = wf.readframes(4000)
                if len(data) == 0:
                    break
                if rec.AcceptWaveform(data):
                    result = json.loads(rec.Result())
                    transcription += result.get("text", "") + " "
            
            # Get final result
            result = json.loads(rec.FinalResult())
            transcription += result.get("text", "")
            
            end_time = time.time()
            wf.close()
            
            return transcription.strip(), end_time - start_time
        except Exception as e:
            print(f"Vosk transcription error: {e}")
            return "", 0.0
    
    def giga_transcribe(self, audio_path):
        """Transcribe using GigaAM"""
        start_time = time.time()
        
        try:
            transcription = self.giga_model.transcribe(audio_path)
                      
            end_time = time.time()
            
            return transcription, end_time - start_time
        except Exception as e:
            print(f"GigaAM transcription error: {e}")
            return "", 0.0
    
    def calculate_rtf(self, processing_time, audio_duration):
        """Calculate Real Time Factor"""
        if audio_duration == 0:
            return 0.0
        return processing_time / audio_duration
    
    def get_audio_duration(self, audio_path):
        """Get audio duration in seconds"""
        try:
            with wave.open(audio_path, 'rb') as wf:
                frames = wf.getnframes()
                rate = wf.getframerate()
                return frames / float(rate)
        except:
            # Fallback using librosa
            audio, sr = librosa.load(audio_path, sr=None)
            return len(audio) / sr
    
    def run_comparison(self, audio_dir, reference_dir):
        """Run comparison on all audio files"""
        results = []
        
        audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]
        
        if not audio_files:
            print(f"No WAV files found in {audio_dir}")
            return results
        
        print(f"Found {len(audio_files)} audio files for comparison")
        
        for audio_file in audio_files:
            audio_path = os.path.join(audio_dir, audio_file)
            ref_path = os.path.join(reference_dir, audio_file.replace('.wav', '.txt'))
            
            # Check if reference file exists
            if not os.path.exists(ref_path):
                print(f"Reference file {ref_path} not found, skipping")
                continue
            
            # Load reference text
            with open(ref_path, 'r', encoding='utf-8') as f:
                reference_text = f.read().strip()
            
            audio_duration = self.get_audio_duration(audio_path)
            print(f"Audio duration: {audio_duration:.2f} seconds")
            print(f"Reference text: {reference_text}")
            
            # Whisper without prompt
            print("Transcribing with Whisper...")
            whisper_text, whisper_time = self.whisper_transcribe(audio_path)
            whisper_wer, whisper_cer = self.calculate_wer_cer(reference_text, whisper_text)
            whisper_rtf = self.calculate_rtf(whisper_time, audio_duration)
            print(f"Whisper result: {whisper_text}")
            
            # Vosk
            print("Transcribing with Vosk...")
            vosk_text, vosk_time = self.vosk_transcribe(audio_path)
            vosk_wer, vosk_cer = self.calculate_wer_cer(reference_text, vosk_text)
            vosk_rtf = self.calculate_rtf(vosk_time, audio_duration)
            print(f"Vosk result: {vosk_text}")
            
            # GigaAM
            print("Transcribing with GigaAM...")
            giga_text, giga_time = self.giga_transcribe(audio_path)
            giga_wer, giga_cer = self.calculate_wer_cer(reference_text, giga_text)
            giga_rtf = self.calculate_rtf(giga_time, audio_duration)
            print(f"GigaAM result: {giga_text}")
            
            results.append({
                'file': audio_file,
                'duration': audio_duration,
                'reference': reference_text,
                'whisper': {
                    'text': whisper_text, 
                    'wer': whisper_wer, 
                    'cer': whisper_cer, 
                    'rtf': whisper_rtf,
                    'time': whisper_time
                },
                'vosk': {
                    'text': vosk_text,
                    'wer': vosk_wer, 
                    'cer': vosk_cer, 
                    'rtf': vosk_rtf,
                    'time': vosk_time
                },
                'giga': {
                    'text': giga_text,
                    'wer': giga_wer, 
                    'cer': giga_cer, 
                    'rtf': giga_rtf,
                    'time': giga_time
                }
            })
            
            print(f"Processed {audio_file}")
            print(f"Whisper - WER: {whisper_wer:.3f}, CER: {whisper_cer:.3f}, RTF: {whisper_rtf:.3f}")
            print(f"Vosk - WER: {vosk_wer:.3f}, CER: {vosk_cer:.3f}, RTF: {vosk_rtf:.3f}")
            print(f"GigaAM - WER: {giga_wer:.3f}, CER: {giga_cer:.3f}, RTF: {giga_rtf:.3f}")
            print("-" * 50)
        
        return results
    
    def evaluate_prompt_impact(self, domain_audio_dir, domain_terms):
        """Evaluate the impact of bias prompt in Whisper"""
        prompt = " ".join(domain_terms)
        print(f"Using prompt: {prompt}")
        
        audio_files = [f for f in os.listdir(domain_audio_dir) if f.endswith('.wav')]
        results = []
        
        if not audio_files:
            print(f"No WAV files found in {domain_audio_dir}")
            return results
        
        for audio_file in audio_files:
            audio_path = os.path.join(domain_audio_dir, audio_file)
            ref_path = os.path.join(domain_audio_dir, audio_file.replace('.wav', '.txt'))
            
            if not os.path.exists(ref_path):
                print(f"Reference file {ref_path} not found, skipping")
                continue
            
            # Load reference text
            with open(ref_path, 'r', encoding='utf-8') as f:
                reference_text = f.read().strip()
            
            print(f"\nDomain audio: {audio_file}")
            print(f"Reference: {reference_text}")
            
            # Without prompt
            text_no_prompt, time_no_prompt = self.whisper_transcribe(audio_path)
            wer_no_prompt, cer_no_prompt = self.calculate_wer_cer(reference_text, text_no_prompt)
            print(f"No prompt result: {text_no_prompt}")
            
            # With prompt
            text_with_prompt, time_with_prompt = self.whisper_transcribe(audio_path, prompt)
            wer_with_prompt, cer_with_prompt = self.calculate_wer_cer(reference_text, text_with_prompt)
            print(f"With prompt result: {text_with_prompt}")
            
            results.append({
                'file': audio_file,
                'reference': reference_text,
                'no_prompt': {
                    'text': text_no_prompt,
                    'wer': wer_no_prompt, 
                    'cer': cer_no_prompt,
                    'time': time_no_prompt
                },
                'with_prompt': {
                    'text': text_with_prompt,
                    'wer': wer_with_prompt, 
                    'cer': cer_with_prompt,
                    'time': time_with_prompt
                },
                'improvement_wer': wer_no_prompt - wer_with_prompt,
                'improvement_cer': cer_no_prompt - cer_with_prompt
            })
            
            print(f"No prompt - WER: {wer_no_prompt:.3f}, CER: {cer_no_prompt:.3f}")
            print(f"With prompt - WER: {wer_with_prompt:.3f}, CER: {cer_with_prompt:.3f}")
            print(f"Improvement - WER: {results[-1]['improvement_wer']:.3f}, CER: {results[-1]['improvement_cer']:.3f}")
            print("=" * 80)
        
        return results
    
    
    def create_summary_table(self, results):
        """Create a comprehensive summary table of results"""
        if not results:
            print("No results to summarize")
            return None
        
        summary_data = []
        
        for result in results:
            summary_data.append({
                'File': result['file'],
                'Duration': f"{result['duration']:.2f}s",
                'Whisper_WER': f"{result['whisper']['wer']:.3f}",
                'Whisper_CER': f"{result['whisper']['cer']:.3f}",
                'Whisper_RTF': f"{result['whisper']['rtf']:.3f}",
                'Vosk_WER': f"{result['vosk']['wer']:.3f}",
                'Vosk_CER': f"{result['vosk']['cer']:.3f}",
                'Vosk_RTF': f"{result['vosk']['rtf']:.3f}",
                'GigaAM_WER': f"{result['giga']['wer']:.3f}",
                'GigaAM_CER': f"{result['giga']['cer']:.3f}",
                'GigaAM_RTF': f"{result['giga']['rtf']:.3f}",
            })
        
        df = pd.DataFrame(summary_data)
        
        # Add averages row
        avg_row = {
            'File': 'AVERAGE',
            'Duration': '-',
            'Whisper_WER': f"{np.mean([r['whisper']['wer'] for r in results]):.3f}",
            'Whisper_CER': f"{np.mean([r['whisper']['cer'] for r in results]):.3f}",
            'Whisper_RTF': f"{np.mean([r['whisper']['rtf'] for r in results]):.3f}",
            'Vosk_WER': f"{np.mean([r['vosk']['wer'] for r in results]):.3f}",
            'Vosk_CER': f"{np.mean([r['vosk']['cer'] for r in results]):.3f}",
            'Vosk_RTF': f"{np.mean([r['vosk']['rtf'] for r in results]):.3f}",
            'GigaAM_WER': f"{np.mean([r['giga']['wer'] for r in results]):.3f}",
            'GigaAM_CER': f"{np.mean([r['giga']['cer'] for r in results]):.3f}",
            'GigaAM_RTF': f"{np.mean([r['giga']['rtf'] for r in results]):.3f}",
        }
        
        df_avg = pd.DataFrame([avg_row])
        df = pd.concat([df, df_avg], ignore_index=True)
        
        return df

    def create_prompt_summary_table(self, prompt_results):
        """Create summary table for prompt impact results"""
        if not prompt_results:
            print("No prompt results to summarize")
            return None
        
        summary_data = []
        
        for result in prompt_results:
            summary_data.append({
                'File': result['file'],
                'Reference': result['reference'][:50] + '...' if len(result['reference']) > 50 else result['reference'],
                'NoPrompt_WER': f"{result['no_prompt']['wer']:.3f}",
                'NoPrompt_CER': f"{result['no_prompt']['cer']:.3f}",
                'WithPrompt_WER': f"{result['with_prompt']['wer']:.3f}",
                'WithPrompt_CER': f"{result['with_prompt']['cer']:.3f}",
                'WER_Improvement': f"{result['improvement_wer']:.3f}",
                'CER_Improvement': f"{result['improvement_cer']:.3f}",
            })
        
        df = pd.DataFrame(summary_data)
        
        # Add averages row
        avg_row = {
            'File': 'AVERAGE',
            'Reference': '-',
            'NoPrompt_WER': f"{np.mean([r['no_prompt']['wer'] for r in prompt_results]):.3f}",
            'NoPrompt_CER': f"{np.mean([r['no_prompt']['cer'] for r in prompt_results]):.3f}",
            'WithPrompt_WER': f"{np.mean([r['with_prompt']['wer'] for r in prompt_results]):.3f}",
            'WithPrompt_CER': f"{np.mean([r['with_prompt']['cer'] for r in prompt_results]):.3f}",
            'WER_Improvement': f"{np.mean([r['improvement_wer'] for r in prompt_results]):.3f}",
            'CER_Improvement': f"{np.mean([r['improvement_cer'] for r in prompt_results]):.3f}",
        }
        
        df_avg = pd.DataFrame([avg_row])
        df = pd.concat([df, df_avg], ignore_index=True)
        
        return df
    

In [11]:
def main():
    comparator = ASRComparator()
    
    # Part 1: Compare ASR engines
    print("=== ASR Engines Comparison ===")
    main_results = comparator.run_comparison("audio_files", "reference_texts")
    
    # Create and display summary table
    if main_results:
        summary_df = comparator.create_summary_table(main_results)
        print("\n" + "="*100)
        print("SUMMARY TABLE - ASR ENGINES COMPARISON")
        print("="*100)
        print(summary_df.to_string(index=False))
        print("="*100)
        
        # Save to CSV
        summary_df.to_csv("/results/asr_comparison_results.csv", index=False)
        print("\nResults saved to 'asr_comparison_results.csv'")
    
    # Part 2: Evaluate prompt impact
    print("\n\n=== Prompt Impact Evaluation ===")
    domain_terms = ["специфический", "терминология", "доменная", "контекст", "профессиональный"]
    prompt_results = comparator.evaluate_prompt_impact("domain_audio", domain_terms)
    
    # Create and display prompt summary table
    if prompt_results:
        prompt_summary_df = comparator.create_prompt_summary_table(prompt_results)
        print("\n" + "="*100)
        print("SUMMARY TABLE - PROMPT IMPACT EVALUATION")
        print("="*100)
        print(prompt_summary_df.to_string(index=False))
        print("="*100)
        
        # Save to CSV
        prompt_summary_df.to_csv("/results/prompt_impact_results.csv", index=False)
        print("\nPrompt results saved to 'prompt_impact_results.csv'")


main()

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-ru-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-ru-0.22/graph/HCLr.fst vosk-model-small-ru-0.22/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo vosk-model-small-ru-0.22/graph/phones/word_boundary.int


Vosk model loaded successfully!
Loading GigaAM model...


GigaAM model loaded successfully!
=== ASR Engines Comparison ===
No WAV files found in audio_files


=== Prompt Impact Evaluation ===
Using prompt: специфический терминология доменная контекст профессиональный
No WAV files found in domain_audio
